Read and Tweet!
=================

This project has been inspired by - 

* Justin Blinder
* http://projects.justinblinder.com/We-Read-We-Tweet

# Project Description
“We Read, We Tweet” geographically visualizes the dissemination of New York Times articles through Twitter. Each line connects the location of a tweet to the contextual location of the New York Times article it referenced. The lines are generated in a sequence based on the time in which a tweet occurs. The project explores digital news distribution in a temporal and spatial context through the social space of Twitter.

In [1]:
## all imports
from IPython.display import HTML
import numpy as np
import urllib2
import bs4 #this is beautiful soup
import time
import operator
import socket
import cPickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


### Start Scraping

In [4]:
# Scraping the wikipedia page for  'Web Scraping" topic
url = "http://www.crummy.com/software/BeautifulSoup"
source = urllib2.urlopen(url).read()


### Test some functions on the data before moving forward


In [ ]:

# check occurences of 'scraping' in the article
print 'scraping' in source
print source.count('scraping')
position = source.find('Legal issues')
print position

# Beautiful Soup
Using this library for an efficient and easy web scraping in python.


In [5]:
# getting bs4 object

soup = bs4.BeautifulSoup(source)
print soup.prettify()

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/transitional.dtd">
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Beautiful Soup: We called him Tortoise because he taught us.
  </title>
  <link href="mailto:leonardr@segfault.org" rev="made"/>
  <link href="/nb/themes/Default/nb.css" rel="stylesheet" type="text/css"/>
  <meta content="Beautiful Soup: a library designed for screen-scraping HTML and XML." name="Description"/>
  <meta content="Markov Approximation 1.4 (module: leonardr)" name="generator"/>
  <meta content="Leonard Richardson" name="author"/>
 </head>
 <body alink="red" bgcolor="white" link="blue" text="black" vlink="660066">
  <img align="right" src="10.1.jpg" width="250"/>
  <br/>
  <p>
   You didn't write that awful page. You're just trying to get some
data out of it. Beautiful Soup is here to help. Since 2004, it's been
saving programmers hours or days of work on quick-tur

In [30]:
# Testing some random bs4 functions 

first_tag = soup.find('a') # just one occurence
#soup.findAll("a") # all occurences
first_tag.get('href')

link_list = [l.get('href') for l in soup.findAll('a')]
# link_list

## just getting all the links that are external by searching for links starting with http

# test = link_list[5]
# test[0:4]

external_links = []

for l in link_list:
    if l is not None and l[0:4] == 'http':
        external_links.append(l)
        
external_links

['https://code.launchpad.net/beautifulsoup',
 'https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup',
 'http://www.candlemarkandgleam.com/shop/constellation-games/',
 'http://constellation.crummy.com/Constellation%20Games%20excerpt.html',
 'https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup',
 'https://bugs.launchpad.net/beautifulsoup/',
 'http://lxml.de/',
 'http://code.google.com/p/html5lib/',
 'http://www.crummy.com/software/BeautifulSoup/bs3/documentation.html',
 'http://www.nytimes.com/2007/10/25/arts/design/25vide.html',
 'https://github.com/reddit/reddit/blob/85f9cff3e2ab9bb8f19b96acd8da4ebacc079f04/r2/r2/lib/media.py',
 'http://www.harrowell.org.uk/viktormap.html',
 'http://svn.python.org/view/tracker/importer/',
 'http://www2.ljworld.com/',
 'http://www.b-list.org/weblog/2010/nov/02/news-done-broke/',
 'http://esrl.noaa.gov/gsd/fab/',
 'http://laps.noaa.gov/topograbber/',
 'http://groups.google.com/group/beautifulsoup/',
 'https://launchpad.net/beauti

In [44]:
root_node = soup.html
head = root_node.contents[0]
body = root_node.contents[3]
body

<body alink="red" bgcolor="white" link="blue" text="black" vlink="660066">
<img align="right" src="10.1.jpg" width="250"/><br/>
<p>You didn't write that awful page. You're just trying to get some
data out of it. Beautiful Soup is here to help. Since 2004, it's been
saving programmers hours or days of work on quick-turnaround
screen scraping projects.</p>
<div align="center">
<a href="bs4/download/"><h1>Beautiful Soup</h1></a>
<p>"A tremendous boon." -- Python411 Podcast</p>
<p>[ <a href="#Download">Download</a> | <a href="bs4/doc/">Documentation</a> | <a href="#HallOfFame">Hall of Fame</a> | <a href="https://code.launchpad.net/beautifulsoup">Source</a> | <a href="https://groups.google.com/forum/?fromgroups#!forum/beautifulsoup">Discussion group</a> ]</p>
<small>If Beautiful Soup has saved you a lot of time and money, the
best way to pay me back is to check out <a href="http://www.candlemarkandgleam.com/shop/constellation-games/"><i>Constellation
Games</i>, my sci-fi novel about alien v

In [45]:
sourc = "https://soic-indiana-csm.symplicity.com/students/index.php?_ksl=1&s=event&ss=cf&mode=form&id=481a8052e9d4a5ee06c4d6a42c2ccc2e&__paging=0"
scrape = bs4.BeautifulSoup(sourc)

/Users/Abhishek/anaconda/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "https://soic-indiana-csm.symplicity.com/students/index.php?_ksl=1&s=event&ss=cf&mode=form&id=481a8052e9d4a5ee06c4d6a42c2ccc2e&__paging=0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
